In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import os

## Aplicar o modelo SVM (SUPORT VECTOR MACHINE) para classificação de imagens

Será o modelo baseline antes aplicar a rede neural, apenas as imagem com size do flatten <= 100

In [3]:
current_path = os.getcwd()
candidate_path = os.path.join(current_path, "CANDIDATE")
confirmed_path = os.path.join(current_path, "CONFIRMED")
false_positive_path = os.path.join(current_path, "FALSE POSITIVE")

## Lendo as imagens e preparando os dados

In [4]:
candidate_flatten = []

for filename in os.listdir(candidate_path):
    file = os.path.join(candidate_path, filename)
    if os.path.isfile(file):
        img = np.load(file)
        loaded_img = img["matrix_img"]
        flatten_img = loaded_img.flatten()
        if len(flatten_img) <= 100:
            candidate_flatten.append(flatten_img)

In [5]:
len(candidate_flatten)

471

In [6]:
confirmed_flatten = []

for filename in os.listdir(confirmed_path):
    file = os.path.join(confirmed_path, filename)
    if os.path.isfile(file):
        img = np.load(file)
        loaded_img = img["matrix_img"]
        flatten_img = loaded_img.flatten()
        if len(flatten_img) <= 100:
            confirmed_flatten.append(flatten_img)

In [7]:
len(confirmed_flatten)

172

In [8]:
false_positive_flatten = []

for filename in os.listdir(false_positive_path):
    file = os.path.join(false_positive_path, filename)
    if os.path.isfile(file):
        img = np.load(file)
        loaded_img = img["matrix_img"]
        flatten_img = loaded_img.flatten()
        if len(flatten_img) <= 100:
            false_positive_flatten.append(flatten_img)

In [9]:
len(false_positive_flatten)

157

## Juntando as imagens em um único dataframe

In [21]:
[len(a) for a in candidate_flatten]

[81,
 56,
 100,
 99,
 42,
 90,
 90,
 90,
 90,
 90,
 81,
 100,
 42,
 35,
 56,
 90,
 56,
 100,
 90,
 80,
 90,
 72,
 56,
 90,
 72,
 72,
 90,
 72,
 80,
 100,
 80,
 81,
 99,
 90,
 80,
 56,
 100,
 42,
 72,
 42,
 63,
 90,
 72,
 72,
 80,
 99,
 90,
 100,
 56,
 81,
 42,
 72,
 80,
 81,
 36,
 99,
 72,
 56,
 81,
 100,
 42,
 90,
 48,
 49,
 99,
 99,
 99,
 56,
 99,
 81,
 42,
 100,
 90,
 100,
 42,
 90,
 90,
 42,
 90,
 64,
 90,
 72,
 72,
 42,
 81,
 56,
 100,
 56,
 56,
 90,
 56,
 90,
 72,
 90,
 99,
 99,
 42,
 42,
 56,
 90,
 99,
 100,
 99,
 100,
 100,
 80,
 99,
 90,
 42,
 72,
 80,
 100,
 42,
 42,
 64,
 80,
 80,
 100,
 90,
 72,
 42,
 42,
 56,
 100,
 56,
 63,
 100,
 99,
 72,
 81,
 42,
 100,
 90,
 42,
 100,
 72,
 48,
 99,
 72,
 72,
 64,
 90,
 90,
 56,
 56,
 99,
 72,
 63,
 80,
 56,
 90,
 56,
 90,
 42,
 42,
 64,
 99,
 56,
 90,
 100,
 81,
 42,
 81,
 72,
 72,
 72,
 81,
 81,
 48,
 42,
 48,
 64,
 42,
 72,
 90,
 64,
 81,
 42,
 42,
 99,
 63,
 100,
 48,
 100,
 64,
 72,
 42,
 72,
 56,
 81,
 90,
 100,
 63,
 100,
 90,
 

In [22]:
df_candidate = pd.DataFrame(candidate_flatten)
df_candidate["label"] = 2
df_candidate = df_candidate.dropna()
df_candidate.shape

(4, 101)

In [23]:
df_candidate = df_candidate.sample(n=100, replace=True)
df_candidate.shape

(100, 101)

In [ ]:

df_candidate = df_candidate.reset_index(drop=True)

In [10]:
df_candidate.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,label
0,-0.987579,2.768197,5.893777,2.320225,14.188142,36.667847,31.775967,5.464093,-1.807136,0.041990,...,13.852238,8.683250,15.929841,13.540003,5.987167,0.623571,-0.961029,-0.932092,0.226252,2
1,-1.333192,1.520788,-31.330189,-33.574200,34.586807,110.374222,-14.511077,-29.818491,-19.623732,-13.179517,...,27.098289,437.558167,1942.102173,4888.783203,3891.703369,1315.619019,372.269745,95.591873,12.411943,2
2,0.604086,14.170194,8.591640,1.486126,1.569907,5.878654,3.366610,1.869908,-0.468823,-0.493468,...,2.633413,1.416048,3.190238,24.451214,43.047840,18.847809,7.712321,2.344949,5.888833,2
3,-1.333192,1.520788,-31.330189,-33.574200,34.586807,110.374222,-14.511077,-29.818491,-19.623732,-13.179517,...,27.098289,437.558167,1942.102173,4888.783203,3891.703369,1315.619019,372.269745,95.591873,12.411943,2
4,1.245763,1.462209,-0.971463,12.630104,-0.817625,4.208768,8.796954,7.050017,7.955020,4.323209,...,0.840679,3.930473,14.043644,14.701113,23.758627,4.797647,1.470763,2.476656,0.068132,2


In [11]:
df_confirmed = pd.DataFrame(confirmed_flatten)
df_confirmed["label"] = 1
df_confirmed = df_confirmed.dropna()
df_confirmed = df_confirmed.sample(n=100, replace=True)
df_confirmed = df_confirmed.reset_index(drop=True)

In [12]:
df_confirmed.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,label
0,7.433475,11.070701,18.107439,20.617672,28.901659,14.036651,7.514658,5.697033,7.169289,12.232132,...,1.870897,6.752622,4.292672,20.104425,9.921350,6.922132,10.920563,2.852748,11.578664,1
1,1.313727,2.565219,5.195151,4.972635,11.155852,7.777151,0.981240,-0.783915,-0.092803,0.455671,...,11.083520,1.527252,1.592452,3.538505,15.563698,24.686052,15.002617,1.923006,1.895209,1
2,7.433475,11.070701,18.107439,20.617672,28.901659,14.036651,7.514658,5.697033,7.169289,12.232132,...,1.870897,6.752622,4.292672,20.104425,9.921350,6.922132,10.920563,2.852748,11.578664,1
3,14.581443,54.842793,44.986202,44.747704,40.213253,33.742290,-2.099796,2.138617,2.298703,-3.186992,...,5.457812,5.587168,14.880466,46.609009,86.456009,69.805313,29.325153,14.599321,5.247704,1
4,2.596646,7.783507,5.547736,8.774408,22.451269,25.834993,15.342981,7.678573,9.571696,8.759951,...,2.277848,2.844331,6.532720,12.255380,16.746836,6.611332,10.611326,13.384288,12.085614,1


In [13]:
df_false_positive = pd.DataFrame(false_positive_flatten)
df_false_positive["label"] = 0
df_false_positive = df_false_positive.dropna()
df_false_positive = df_false_positive.sample(n=100, replace=True)
df_false_positive = df_false_positive.reset_index(drop=True)

In [14]:
df_false_positive.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,label
0,5.512897,7.982657,17.182034,50.143063,79.868851,30.17564,7.807992,5.600437,-1.407505,5.434684,...,2.980541,4.018745,2.273421,11.186778,17.114073,18.383621,17.492125,13.74321,7.695585,0
1,5.512897,7.982657,17.182034,50.143063,79.868851,30.17564,7.807992,5.600437,-1.407505,5.434684,...,2.980541,4.018745,2.273421,11.186778,17.114073,18.383621,17.492125,13.74321,7.695585,0
2,5.512897,7.982657,17.182034,50.143063,79.868851,30.17564,7.807992,5.600437,-1.407505,5.434684,...,2.980541,4.018745,2.273421,11.186778,17.114073,18.383621,17.492125,13.74321,7.695585,0
3,5.512897,7.982657,17.182034,50.143063,79.868851,30.17564,7.807992,5.600437,-1.407505,5.434684,...,2.980541,4.018745,2.273421,11.186778,17.114073,18.383621,17.492125,13.74321,7.695585,0
4,5.512897,7.982657,17.182034,50.143063,79.868851,30.17564,7.807992,5.600437,-1.407505,5.434684,...,2.980541,4.018745,2.273421,11.186778,17.114073,18.383621,17.492125,13.74321,7.695585,0


In [15]:
## concatenate all dataframes, shuffle the data and reset index

df = pd.concat([df_candidate, df_confirmed, df_false_positive])
df = df.sample(frac=1).reset_index(drop=True)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,label
0,5.512897,7.982657,17.182034,50.143063,79.868851,30.175640,7.807992,5.600437,-1.407505,5.434684,...,2.980541,4.018745,2.273421,11.186778,17.114073,18.383621,17.492125,13.743210,7.695585,0
1,-0.987579,2.768197,5.893777,2.320225,14.188142,36.667847,31.775967,5.464093,-1.807136,0.041990,...,13.852238,8.683250,15.929841,13.540003,5.987167,0.623571,-0.961029,-0.932092,0.226252,2
2,1.313727,2.565219,5.195151,4.972635,11.155852,7.777151,0.981240,-0.783915,-0.092803,0.455671,...,11.083520,1.527252,1.592452,3.538505,15.563698,24.686052,15.002617,1.923006,1.895209,1
3,0.604086,14.170194,8.591640,1.486126,1.569907,5.878654,3.366610,1.869908,-0.468823,-0.493468,...,2.633413,1.416048,3.190238,24.451214,43.047840,18.847809,7.712321,2.344949,5.888833,2
4,7.433475,11.070701,18.107439,20.617672,28.901659,14.036651,7.514658,5.697033,7.169289,12.232132,...,1.870897,6.752622,4.292672,20.104425,9.921350,6.922132,10.920563,2.852748,11.578664,1


In [16]:
df.label.value_counts()

label
0    100
2    100
1    100
Name: count, dtype: int64

In [17]:
# all the first 100 columns are the pixel values, the last column is the label

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [18]:
y.value_counts()

label
0    100
2    100
1    100
Name: count, dtype: int64

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

In [29]:
# import random forest
from sklearn.ensemble import RandomForestClassifier


In [32]:
# clf = make_pipeline(StandardScaler(), SVC(gamma="auto"))
rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, criterion="entropy")
rf.fit(X_train, y_train)
# clf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=2, random_state=0)

In [33]:
# print(clf.score(X_test, y_test))
print(rf.score(X_test, y_test))

1.0


In [27]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        90
           1       1.00      1.00      1.00        92
           2       1.00      1.00      1.00        88

    accuracy                           1.00       270
   macro avg       1.00      1.00      1.00       270
weighted avg       1.00      1.00      1.00       270



In [28]:
from sklearn.model_selection import cross_val_score

# Using the entire dataset (X, y) before splitting
scores = cross_val_score(clf, X, y, cv=5)  # 5-fold cross-validation
print("Accuracy scores for each fold:", scores)
print("Mean cross-validation accuracy:", scores.mean())


Accuracy scores for each fold: [1. 1. 1. 1. 1.]
Mean cross-validation accuracy: 1.0
